In [121]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay, classification_report, accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gyova\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [122]:
df = pd.read_csv('../data/train.csv')
df = df.drop(columns=['Unnamed: 0'])

df.isnull().any()

label                False
preprocessed_news    False
dtype: bool

In [123]:
stop_words = set(stopwords.words('portuguese'))

def remover_stop_words(news):
    palavras = news.split()
    palavras_sem_stop = [palavra for palavra in palavras if palavra.lower() not in stop_words]
    return ' '.join(palavras_sem_stop)

def review_cleaning(text):
    
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

df["preprocessed_news"] = df["preprocessed_news"].apply(remover_stop_words)
df["preprocessed_news"] = df["preprocessed_news"].apply(review_cleaning)


In [124]:
df['label'] = df.apply(lambda row: 0 if row.label == 'fake' else 1, axis=1)

X_train = df.drop(['label'], axis = 1)
Y_train = df['label']

In [125]:
vocab_length = len(X_train) + 1
maxlen=256

In [119]:

X_train = X_train['preprocessed_news'].apply(lambda x: x.lower())
train_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>',num_words=8000)
train_tokenizer.fit_on_texts(X_train.values)
train_word_index = train_tokenizer.word_index
train_sequences = train_tokenizer.texts_to_sequences(X_train)
vocab_length = len(train_word_index) + 1
maxlen=8000
train_padded_seqeunces = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=maxlen)


vectorizer = CountVectorizer(max_features=8000)
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train).toarray()

In [126]:
print(f'Vocab length {vocab_length}, Train sequences {len(train_sequences)}, Test sequences {len(test_sequences)} ')


Vocab length 4609, Train sequences 4608, Test sequences 1152 


##### Logistic Regression


In [ ]:
lr_classifier = LogisticRegression()
lr_classifier.fit(X_train,Y_train)
lr_classifier_train = round(lr_classifier.score(X_train, Y_train) * 100, 2)

with open('../models/logisticRegression.pkl', 'wb') as arquivo:
    pickle.dump(lr_classifier, arquivo)

##### Multilayer perceptron (MLP)

In [ ]:
mlp = MLPClassifier(random_state=1)
mlp.fit(X_train,Y_train)
mlp_train = round(mlp.score(X_train, Y_train) * 100, 2)

with open('../models/MLPClassifier.pkl', 'wb') as arquivo:
    pickle.dump(mlp, arquivo)


##### Multilayer perceptron (MLP) Com GridSearchCV

In [ ]:
parameters = {'solver': ['sgd', 'lbfgs'],
              'hidden_layer_sizes':(20, 10,4), 
              'random_state':[2],
              'activation': ['tanh', 'relu'],
              'max_iter': [2000000],
              "batch_size" : ["auto"],
              "learning_rate_init": [0.004],
              "validation_fraction":[0.1],
              "learning_rate": ['constant']
              }

clf = GridSearchCV(MLPClassifier(), parameters, cv=2)
epochs_hist = clf.fit(X_train,Y_train)
mlpG_train = round(clf.score(X_train, Y_train) * 100, 2)


with open('../models/MLPClassifierWithGridSearchCV.pkl', 'wb') as arquivo:
    pickle.dump(clf, arquivo)



#### Decision Tree

In [ ]:
decisionTree = DecisionTreeClassifier()
decisionTree.fit(X_train,Y_train)
decisionTree_train = round(clf.score(X_train, Y_train) * 100, 2)

with open('../models/decisionTree.pkl', 'wb') as arquivo:
    pickle.dump(decisionTree, arquivo)

#### RNN

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

output_dim = 64
epochs = 100
tf.keras.backend.clear_session()

modelSimpleRNN = tf.keras.models.Sequential()
modelSimpleRNN.add(tf.keras.layers.Embedding(vocab_length, output_dim, input_length=maxlen))
modelSimpleRNN.add(tf.keras.layers.GRU(60, activation='tanh', return_sequences=True))
modelSimpleRNN.add(tf.keras.layers.Conv1D(30, 3, activation='relu'))
modelSimpleRNN.add(tf.keras.layers.LSTM(30, return_sequences=True))
modelSimpleRNN.add(tf.keras.layers.SimpleRNN(3, activation='tanh'))
modelSimpleRNN.add(tf.keras.layers.Dropout(0.25))
modelSimpleRNN.add(tf.keras.layers.Dense(2, activation='relu'))
modelSimpleRNN.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))
modelSimpleRNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

historySimpleRNN = modelSimpleRNN.fit(train_padded_seqeunces, Y_train, epochs=epochs, validation_split=0.3, callbacks=[early_stopping])
rnn_train = round(historySimpleRNN.history['accuracy'][-1] * 100, 2)
modelSimpleRNN.save('../models/pc/modelo_rnn.keras', save_format='tf')

plt.plot(historySimpleRNN.history['loss'])
plt.title('Model loss progress during training')
plt.xlabel('Epoch')
plt.ylabel('Training loss')
plt.legend(['Training loss'])
plt.plot(historySimpleRNN.history['accuracy'])
plt.title('Model accuracy progress during training')
plt.xlabel('Epoch')
plt.ylabel('Training accuracy')
plt.legend(['Training accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

output_dim = 64
epochs = 100
tf.keras.backend.clear_session()

modelSimpleRNN = tf.keras.models.Sequential()
modelSimpleRNN.add(tf.keras.layers.Embedding(vocab_length, output_dim, input_length=maxlen))
modelSimpleRNN.add(tf.keras.layers.GRU(60, activation='tanh', return_sequences=True))
modelSimpleRNN.add(tf.keras.layers.Conv1D(30, 3, activation='relu'))
modelSimpleRNN.add(tf.keras.layers.LSTM(30, return_sequences=True))
modelSimpleRNN.add(tf.keras.layers.SimpleRNN(3, activation='tanh'))
modelSimpleRNN.add(tf.keras.layers.Dropout(0.25))
modelSimpleRNN.add(tf.keras.layers.Dense(2, activation='relu'))
modelSimpleRNN.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))
modelSimpleRNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelSimpleRNN.summary()

#### LSTM (BI)

In [ ]:

modelLSTM = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_length, output_dim, input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(10)),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
modelLSTM.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

historyLSTM = modelLSTM.fit(train_padded_seqeunces, Y_train, epochs=epochs, validation_split=0.3, callbacks=[early_stopping])
LSTM_train = round(historyLSTM.history['accuracy'][-1] * 100, 2)
modelLSTM.save('../models/modelLSTM.keras', save_format='tf')

plt.plot(historyLSTM.history['loss'])
plt.title('Model loss progress during training')
plt.xlabel('Epoch')
plt.ylabel('Training loss')
plt.legend(['Training loss'])
plt.plot(historyLSTM.history['accuracy'])
plt.title('Model accuracy progress during training')
plt.xlabel('Epoch')
plt.ylabel('Training accuracy')
plt.legend(['Training accuracy'])

In [ ]:
vocab_length

In [ ]:
test_padded_seqeunces.shape

#### HAN

In [128]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, GRU, LSTM, SimpleRNN, Conv1D, Dense, Dropout, Attention, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    
maxlen=1

print("Shape before reshaping:", train_padded_seqeunces.shape)
x_train_padded_sequences = train_padded_seqeunces[:, :, tf.newaxis]
print("Shape after reshaping:", x_train_padded_sequences.shape)

document_input = Input(shape=(maxlen,), dtype='float32')
word_embedding = Embedding(vocab_length, output_dim, input_length=maxlen)(document_input)
word_lstm = Bidirectional(LSTM(50, return_sequences=True))(word_embedding)
word_attention = Attention()([word_lstm, word_lstm])
word_representation = tf.reduce_sum(word_attention * word_lstm, axis=1)

sentence_input = Input(shape=(None, maxlen), dtype='float32')
sentence_encoder = Bidirectional(LSTM(20, return_sequences=True))(sentence_input)
sentence_attention = Attention()([sentence_encoder, sentence_encoder])
sentence_representation = tf.reduce_sum(sentence_attention * sentence_encoder, axis=1)

document_representation = Dense(2, activation='relu')(sentence_representation)
document_representation = Dropout(0.5)(document_representation)
output = Dense(1, activation='sigmoid')(document_representation)


# Crie e compile o modelo
model_han = Model(inputs=[sentence_input, document_input], outputs=output)
model_han.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Imprima o resumo do modelo
model_han.summary()

historyHAN = model_han.fit([x_train_padded_sequences, x_train_padded_sequences], Y_train, epochs=epochs,batch_size=64, validation_split=0.3, callbacks=[early_stopping])
HAN_train = round(historyHAN.history['accuracy'][-1] * 100, 2)
model_han.save('../models/modelHAN.keras', save_format='tf')


Shape before reshaping: (4608, 8000)
Shape after reshaping: (4608, 8000, 1)
Model: "model_31"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_67 (InputLayer)       [(None, None, 1)]            0         []                            
                                                                                                  
 bidirectional_67 (Bidirect  (None, None, 40)             3520      ['input_67[0][0]']            
 ional)                                                                                           
                                                                                                  
 attention_66 (Attention)    (None, None, 40)             0         ['bidirectional_67[0][0]',    
                                                                     'bidirectional_67[0][0]']    
               

In [ ]:
df_test = pd.read_csv('../data/test.csv')
maxlen=256


stop_words = set(stopwords.words('portuguese'))

def remover_stop_words(news):
    palavras = news.split()
    palavras_sem_stop = [palavra for palavra in palavras if palavra.lower() not in stop_words]
    return ' '.join(palavras_sem_stop)

def review_cleaning(text):
    
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

df_test["preprocessed_news"] = df_test["preprocessed_news"].apply(remover_stop_words)
df_test["preprocessed_news"] = df_test["preprocessed_news"].apply(review_cleaning)

df_test['label'] = df_test.apply(lambda row: 0 if row.label == 'fake' else 1, axis=1)

X_test = df_test['preprocessed_news'].apply(lambda x: x.lower())

test_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
test_tokenizer.fit_on_texts(X_test.values)
test_word_index = test_tokenizer.word_index
test_sequences = test_tokenizer.texts_to_sequences(X_test)
vocab_length = len(test_word_index) + 1
maxlen=256
test_padded_seqeunces = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, padding='post', maxlen=maxlen, truncating='post')

Y_test = df_test['label']

y_test_pred = model_han.predict(test_padded_seqeunces)

y_test_pred = (y_test_pred > 0.70).astype(int)

Y_test = Y_test.astype(int)
Y_test = Y_test.ravel()
y_test_pred = y_test_pred.ravel()
    
han_acc = round(accuracy_score(Y_test, y_test_pred), 2)

cm = confusion_matrix(Y_test, y_test_pred)
print(classification_report(Y_test, y_test_pred))

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Fake','True']).plot()

## Test

In [ ]:
df_test = pd.read_csv('../data/test.csv')

#df_test = pd.read_csv('../data/dataset_0_palavras.csv')
#df_test = df_test.drop(columns=['Unnamed: 0', 'ID', 'author',
#       'average_word_length', 'category', 'date_publication', 'link',
#       'percent_speeling_errors', 'number_words'])

#df_test = df_test.rename(columns={'fake_news': 'label'})
#df_test = df_test.rename(columns={'text': 'preprocessed_news'})

#df_test = df_test.drop(df_test.index[400:], inplace=False)
df_test

In [ ]:
maxlen=256


stop_words = set(stopwords.words('portuguese'))

def remover_stop_words(news):
    palavras = news.split()
    palavras_sem_stop = [palavra for palavra in palavras if palavra.lower() not in stop_words]
    return ' '.join(palavras_sem_stop)

def review_cleaning(text):
    
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

df_test["preprocessed_news"] = df_test["preprocessed_news"].apply(remover_stop_words)
df_test["preprocessed_news"] = df_test["preprocessed_news"].apply(review_cleaning)

df_test['label'] = df_test.apply(lambda row: 0 if row.label == 'fake' else 1, axis=1)

X_test = df_test['preprocessed_news'].apply(lambda x: x.lower())

test_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
test_tokenizer.fit_on_texts(X_test.values)
test_word_index = test_tokenizer.word_index
test_sequences = test_tokenizer.texts_to_sequences(X_test)
vocab_length = len(test_word_index) + 1
maxlen=256
test_padded_seqeunces = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, padding='post', maxlen=maxlen, truncating='post')


vectorizer = CountVectorizer(max_features=8000)
vectorizer.fit(X_test)
X_test = vectorizer.transform(X_test).toarray()
Y_test = df_test['label']


In [ ]:
with open('../models/logisticRegression.pkl', 'rb') as arquivo:
    lr_classifier = pickle.load(arquivo)
with open('../models/MLPClassifier.pkl', 'rb') as arquivo:
    mlp = pickle.load(arquivo)
with open('../models/MLPClassifierWithGridSearchCV.pkl', 'rb') as arquivo:
    mlpG = pickle.load(arquivo)
with open('../models/decisionTree.pkl', 'rb') as arquivo:
    decisionTree = pickle.load(arquivo)
    
rnn = load_model('../models/modelo_rnn.keras')
modelLSTM = load_model('../models/modelLSTM.keras')
model_han = load_model('../models/modelHAN.keras')

##### Logistic Regression


In [ ]:
y_test_pred = lr_classifier.predict(X_test)
y_test_pred = (y_test_pred > 0.75)
lr_classifier_acc = round(accuracy_score(Y_test, y_test_pred) * 100, 2)
cm = confusion_matrix(Y_test, y_test_pred)
print(classification_report(Y_test, y_test_pred))

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Fake','True']).plot()



##### Multilayer perceptron (MLP)

In [ ]:
y_test_pred = mlp.predict(X_test)
y_test_pred = (y_test_pred > 0.75)
mlp_acc = round(accuracy_score(Y_test, y_test_pred) * 100, 2)

cm = confusion_matrix(Y_test, y_test_pred)
print(classification_report(Y_test, y_test_pred))

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Fake','True']).plot()


##### Multilayer perceptron (MLP) Com GridSearchCV

In [ ]:

y_test_pred = mlpG.predict(X_test)
y_test_pred = (y_test_pred > 0.75)
mlpG_acc = round(accuracy_score(Y_test, y_test_pred) * 100, 2)

cm = confusion_matrix(Y_test, y_test_pred)
print(classification_report(Y_test, y_test_pred))

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Fake','True']).plot()

    

#### Decision Tree

In [ ]:
y_test_pred = decisionTree.predict(X_test)
y_test_pred = (y_test_pred > 0.75)
decisionTree_acc = round(accuracy_score(Y_test, y_test_pred) * 100, 2)

cm = confusion_matrix(Y_test, y_test_pred)
print(classification_report(Y_test, y_test_pred))

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Fake','True']).plot()


#### RNN

In [ ]:

y_test_pred = rnn.predict(test_padded_seqeunces)
y_test_pred = (y_test_pred > 0.70)
rnn_acc = round(accuracy_score(Y_test, y_test_pred) * 100, 2)

cm = confusion_matrix(Y_test, y_test_pred)
print(classification_report(Y_test, y_test_pred))

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Fake','True']).plot()


#### LSTM(BI)

In [ ]:

y_test_pred = modelLSTM.predict(test_padded_seqeunces)
y_test_pred = (y_test_pred > 0.70)
lstm_acc = round(accuracy_score(Y_test, y_test_pred) * 100, 2)

cm = confusion_matrix(Y_test, y_test_pred)
print(classification_report(Y_test, y_test_pred))

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Fake','True']).plot()


HAN

In [ ]:
y_test_pred = model_han.predict(test_padded_seqeunces)

y_test_pred = (y_test_pred > 0.70).astype(int)

# Ajustar a variável Y_test para garantir que seja do tipo inteiro (se necessário)
Y_test = Y_test.astype(int)
han_acc = round(accuracy_score(Y_test, y_test_pred), 2)

cm = confusion_matrix(Y_test, y_test_pred)
print(classification_report(Y_test, y_test_pred))

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Fake','True']).plot()


### Comparando Modelos Diferentes

In [ ]:
model = pd.DataFrame({
    'Model': [
        'Logistic Regression',
        'Decision Tree',
        'MLPClassifier',
        'MLPClassifier with grid',
        'RNN',
        'RNN LSTM(BI)',
        'HAN'
    ],
    'Model Accuracy Score': [
        lr_classifier_acc, decisionTree_acc,
        mlp_acc, mlpG_acc,rnn_acc,lstm_acc,han_acc
    ]
})

In [ ]:
model = model.sort_values('Model Accuracy Score',ascending=False)
model

In [ ]:
df_subset = df_test.drop(df_test.index[1:], inplace=False)
df_subset.to_csv('../data/sub_test.csv', index=False)
df_subset.preprocessed_news = "O número de feminicídios aumentou durante o governo Bolsonaro. Os dados do Anuário Brasileiro de Segurança Pública mostraram que em 2018 último ano antes de ele assumir como presidente o Brasil teve um total de 1.229 casos (página 157). Nos dois primeiros anos do governo Bolsonaro o número aumentou para 1.330 em 2019 e 1.354 em 2020. Em 2021 o número caiu para 1.341 mas seguiu acima do patamar de 2018."

new_row = new_row = pd.DataFrame({'index': [1], 'label': ['0'], 'preprocessed_news': ["Não pega fogo na floresta amazônica. Isso é falácia – Jair Bolsonaro (PL) presidente e candidato à reeleição em sabatina da RedeTV em 1 de setembro de 2022"]})
df_subset = pd.concat([df_subset, new_row], ignore_index=True)
new_row = new_row = pd.DataFrame({'index': [2], 'label': ['1'], 'preprocessed_news': ["Hoje você leva em média um dia para abrir uma empresa – Jair Bolsonaro (PL) presidente e candidato à reeleição em sabatina da RedeTV em 1 de setembro de 2022"]})
df_subset = pd.concat([df_subset, new_row], ignore_index=True)

new_row = new_row = pd.DataFrame({'index': [1], 'label': ['1'], 'preprocessed_news': [",historia fisico minas marte brasileiro ivair gontijo narra livro participacao missao curiosity agosto curiosity pousava marte maior sofisticado veiculo espacial ser enviado sucesso outro planeta radares permitiram pouso momento critico missao projetados equipe liderada fisico brasileiro ivair gontijo pesquisador jet propulsion laboratory jpl agencia espacial americana nasa novo livro caminho marte acaba ser lancado gontijo conta nao apenas detalhes aventura envolveu projeto lancamento pouso historico curiosity tambem singular trajetoria pessoal criado pequena cidade moema margens rio sao francisco interior minas gerais gontijo decidiu trocar emprego fazenda curso fisica universidade federal minas gerais deixando familiares incredulos so comeco antes chegar nasa ainda fez doutorado escocia estados unidos elon musk divulga design atualizado proximo lancador espacial objetivo livro mostrar estudantes brasileiros geral conseguir algo trabalhar nasa alcance maioria pessoas disposta pagar preco preco alto horas trabalho estudo foco planejamento disse gontijo estado autor intercala narrativa propria trajetoria historia estudos sobre marte segundo apaixonado humanidade ha seculos cientistas precisamos tornar ciencia acessivel emocionante populacao porque nenhum pais vai frente muita ciencia tecnologia procuro mostrar ciencia pode ser tao emocionante gol final copa mundo afirmou conseguir emprego nasa segundo gontijo extremamente dificil alem formacao solida anos estudo intenso preciso armar insistencia astronomica nao primeira vez bati porta nasa consegui entrar primeira vez fiz entrevista ofereceram emprego temporario achei nao valia pena contato gerente missoes avisou precisavam alguem perfil deu pessoas laboratorio ninguem nunca respondeu nao desisti conta dia engenheiro jpl ligou dizendo encontrou curriculo site conferencia precisavam pessoa exatamente formacao fiz entrevista nao deu certo preciso esperar ano conto todos detalhes livro disse fisico missao segundo gontijo curiosity veiculo sofisticado ja construido enviado outro planeta tarefa dificultada tamanho veiculo semelhante carro compacto curiosity pesaria quilos terra possui mastro dois metros altura missao anterior envolvia veiculo menor torno quilos tecnologia toda viabilizar pouso curiosity marte nao existia antes tecnologia utilizamos missoes anteriores airbags nao funcionaria nessa circunstancia tinhamos fazer veiculo descer rodas explicou segundo cientista trabalho missao dessas dimensoes exigente tenso momentos criticos lancamento pouso basta falha milhares possiveis perderem anos trabalho bilhoes dolares processo descida marte ficou conhecido sete minutos terror porque veiculo aproximando planeta mil quilometros hora rota colisao queda livre velocidade precisava ser reduzida zero apenas sete minutos contou antes descida capsula continha curiosity perde energia atmosfera marciana usando escudo frontal calor paraquedas utilizado reduzir ainda velocidade descida final ultimos quatro seis quilometros usamos retrofoguetes explica voo controlado retrofoguetes veiculo colocado chao guindaste seguida afasta nao causar danos equipamento correu tudo melhor esperavamos vi colega sala controle dizendo achamos solo radar realmente acreditei daria certo seguida lembro comemoracoes sala controle muitos gritos lagrimas espetacular lembra radares gontijo nao funcionassem curiosity espatifaria superficie marciana tudo estaria perdido colegas diziam radares nao funcionassem todo resto irrelevante porque veiculo viraria monte ferro velho superficie marte projeto custou us bilhoes entao tensao grande acordo gontijo missao curiosity ja terminou oficialmente veiculo continua fazendo pesquisas superficie marciana onde percorreu pouco quilometros ja encontramos leito seco rio fundo lago rochas sedimentares locais onde havia agua ph neutro pessoa poderia beber operadores nasa recebem dados curiosity diariamente ate hoje apos recepcao dados cientificos sao discutidos fotos sao analisadas engenheiros tecnicos cientistas decidem fazer dia seguinte comandos sao programados enviados marte todos dias satelite orbita planeta satelite repassa dados veiculo solo executa comandos coleta dados tarde marciana transmite novos dados direcao ceu marciano onde passara satelite reenvia terra explicou futuro atualmente gontijo trabalhando proxima missao marte mars enviara planeta vermelho veiculo semelhante curiosity novo veiculo tera conjunto diferente instrumentos descera local distinto ideia coletor amostras disse cientista amostras coletadas veiculo mars serao colocadas roboticamente tubos serao selados hermeticamente deixados superficie planeta segunda missao sera enviada coletar amostras serao colocadas pequeno foguete enviadas orbita marte ali amostras vao ficar girando torno marte terceira missao simples barata saira entao terra encontrara amostras orbita marte trara tudo volta terra necessario porque terra laboratorios sofisticados analisar amostras chegam ocupar predio inteiro nenhum instrumento possa ser enviado atualmente marte mesma capacidade analise objetivos exploracao marte segundo gontijo encontrar vestigios vida planeta cientistas vestigio fossil forma primitiva vida bacteria totalmente extinta ja descoberta historica igual talvez maiores descobertas especie humana todos seres vivos terra mesma origem todos virus seres humanos usamos processo codificacao dna vida formar outro planeta sera usaria processo nao sabemos dia descobrirmos vida terra sera primeira coisa ser estudada sao processos formam vida outro planeta descoberta espetacular"]})
df_subset = pd.concat([df_subset, new_row], ignore_index=True)
new_row = new_row = pd.DataFrame({'index': [2], 'label': ['1'], 'preprocessed_news': [",relator lava jato nega pedido lula ser ouvido advogados lula haviam pedido prestasse novo depoimento defesa apresentou nesta novos documentos sobre propriedade triplex tribunal regional federal regiao nao vai ouvir luiz inacio lula silva antes julgamento apelacao marcado janeiro primeira instancia politico condenado anos meses corrupcao passiva lavagem dinheiro processo lava jato envolvendo triplex advogados lula haviam protocolado dois pedidos lula ouvido antes julgamento recurso setembro outro inicio janeiro decisao relator processos lava jato tribunal joao gebran neto diz eventual deferimento reinterrogatorio passa necessariamente apreciacao alegacoes invalidade daquele prestado perante juizo origem alem disso gebran cita jurisprudencias anteriores negaram pedidos semelhantes ressalta faculdade decidir novo depoimento somente juizo nao reu acusacao apelacao defesa tambem pede inocentado acusacao ocultacao propriedade imovel recebido propina empreiteira oas troca favores petrobras defesa moro nao interesse apurar fatos nesta advogados lula protocolaram apelacao novos documentos segundo defesa reforcariam oas proprietaria triplex decisao proferida dezembro juizo vara execucao titulos distrito federal autos processo determinou penhora apartamento condominio solaris guaruja sp satisfacao divida oas matricula atualizada cartorio imoveis municipio tambem incluida peticao documentos defesa reforca propriedade imovel pertence oas nao lula sentenca juiz sergio moro permite recorra liberdade setembro advogados lula ja haviam ingressado pedido relator processos lava jato desembargador joao gebran neto porem segundo assessoria imprensa defesa pedido nao apreciado motivou novo pedido julgamento recurso apresentado defesa lula sera realizado dia janeiro sede tribunal regional regiao porto alegre justica federal parana tambem determinou bloqueio r milhoes estabelecido dano minimo sequestro apartamento lula tambem bloqueados r mil contas bancarias cerca r milhoes depositados dois planos previdencia privada outros dois reus processo tambem condenados quatro absolvidos movimento terra mst havia anunciado acampamento manifestar apoio lula durante julgamento proibido justica realizar protesto dessa maneira despacho publicado dezembro neto proibiu instalacao acampamento movimento trabalhadores rurais terra mst area parque terrenos vizinhos ate tres dias apos julgamento manifestantes poderao acessar area caso desembargadores decidam manter decisao primeira instancia podem determinar prisao lula caso executada moro curitiba decidir so ira prisao apos todos recursos terem sido esgotados supremo tribunal federal stf determinou reu condenado segunda instancia ja comece cumprir pena prisao recorrendo tribunais superiores assunto porem deve voltar ser discutido ministros ainda nao ha data julgamento inelegibilidade lula vez assunto justica eleitoral lei ficha limpa preve condenado segunda instancia caso caso desembargadores confirmem sentenca nao pode candidatar eventual condenacao si so nao influenciaria possivel candidatura neste caso decisao ficaria conta tribunal superior eleitoral tse pode ser acionado ministerio publico eleitoral mpe algum adversario politico ainda decidir iniciativa propria magistrado"]})
df_subset = pd.concat([df_subset, new_row], ignore_index=True)



In [ ]:
df_subset

In [ ]:
df_subset["preprocessed_news"] = df_subset["preprocessed_news"].apply(remover_stop_words)
df_subset["preprocessed_news"] = df_subset["preprocessed_news"].apply(review_cleaning)


X_test = df_subset['preprocessed_news'].apply(lambda x: x.lower())

test_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
test_tokenizer.fit_on_texts(X_test.values)
test_word_index = test_tokenizer.word_index
test_sequences = test_tokenizer.texts_to_sequences(X_test)
vocab_length = len(test_word_index) + 1
maxlen=256
test_padded_seqeunces = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, padding='post', maxlen=maxlen, truncating='post')


vectorizer = CountVectorizer(max_features=8000)
vectorizer.fit(X_test)
X_test = vectorizer.transform(X_test).toarray()
Y_test = df_subset['label']


In [ ]:

y_test_pred = rnn.predict(test_padded_seqeunces)
y_test_pred = (y_test_pred > 0.5).astype(int)

print(y_test_pred)


y_test_pred = modelLSTM.predict(test_padded_seqeunces)
y_test_pred = (y_test_pred > 0.5).astype(int)

print(y_test_pred)


